# Measuring Genuine Item Info

### -----


In [99]:
import datetime
import requests;                                 # import package used to downlaod data from a web URL
import json;
import pandas as pd;
import seaborn as sns;
import matplotlib.pyplot as plt;
import matplotlib;

from pandas.io.json import json_normalize

matplotlib.style.use('ggplot');
%matplotlib inline

In [89]:
with open('API Keys.txt', 'r') as keyfile:
    apiKeys=keyfile.read().split('\n')
   
url = 'http://backpack.tf/api/IGetPrices/v4/?key=' + apiKeys[0];        # web address of data stream

data_stream = requests.get(url,stream=True);                                    # create an iterable data stream object

In [90]:
FILTER = 'prices.1.Tradable.Craftable.0';

rec = data_stream.iter_lines().next().strip() # Necessary before reading data
data = json.loads(rec)            # convert the json record to a python dictionary


# Figured out how to unnest the JSON that was returned.  Right now we simply have some of the data and some front-end work 
# is complete
items = []
i = 0
for result in data['response']['items']:
    prices = [];
    row = json_normalize(data['response']['items'][result])
    
    # prices.X.Tradable.Craftable.Y.currency
    # X = 0: Normal
    #     1: Unique
    #     2: Vintage
    #     3: Genuine
    #     4: Strange
    # Y = priceIndex
    if(FILTER + '.currency' in row):
        items.append(row)
        items[i]['Name'] = result
        items[i].set_index('Name')
        i += 1
        
len(items)

171

In [91]:
df = pd.concat(items)

for column in df.columns:
    if(column.startswith('Name')):
        continue
    if(not column.startswith(FILTER)):
        df = df.drop(column, 1)

df.columns = ['Name', 'Currency', 'Difference', 'Last Update', 'Value', 'Value High']
df = df[df['Currency'] == 'metal']
df

Name Currency  Difference  Last Update  Value  \
0          Robot Chicken Hat    metal      -0.330   1450859809   2.00   
0              Triad Trinket    metal       0.335   1442414299   8.00   
0            Chief Constable    metal       0.280   1438030985   3.00   
0           Glengarry Bonnet    metal       0.670   1452279646   5.00   
0               Camera Beard    metal      -0.340   1436283365   5.66   
0                Cockfighter    metal       0.000   1443116798  11.00   
0     Heroic Companion Badge    metal      -0.500   1439706823   2.00   
0       Faerie Solitaire Pin    metal       0.110   1429457435   1.66   
0                  Concheror    metal       0.670   1437488842   2.00   
0                Champ Stamp    metal      -0.340   1448354658   9.66   
0          Cold War Luchador    metal       1.340   1430925050   5.00   
0      Backbiter's Billycock    metal       0.440   1430925087   5.66   
0             Dadliest Catch    metal      -0.170   1453251685   1.66   
0          Flying Guillotine    metal       0.340   1450258998   2.00   
0                      Menpo    metal      -0.165   1431488273   2.00   
0        Bolgan Family Crest    metal       0.340   1428125269   2.00   
0               Tartan Shade    metal       0.170   1453741880   6.00   
0           Biomech Backpack    metal       0.505   1428960947   6.00   
0              Blind Justice    metal      -0.330   1434082191   2.00   
0              Freedom Staff    metal      -0.835   1431386810   4.33   
0                Apoco-Fists    metal      -1.000   1451714238   5.00   
0                 Scrap Pack    metal       0.330   1453741893   5.33   
0                 Merc Medal    metal      -0.330   1442077960   5.00   
0                 Last Straw    metal      -0.170   1435298585   3.66   
0         Cross-Comm Express    metal       0.340   1453742040  13.00   
0                  Ham Shank    metal       0.170   1427971841   2.00   
0           Human Cannonball    metal       0.330   1453760940   4.00   
0                    Pilotka    metal       0.110   1427749592   1.66   
0             Sun-on-a-Stick    metal      -0.340   1438454022   1.66   
0              Stainless Pot    metal      -0.330   1450363255   5.33   
..                       ...      ...         ...          ...    ...   
0          Sydney Straw Boat    metal      -0.170   1435506362   1.66   
0      Siberian Sophisticate    metal       0.165   1453741802   4.66   
0                    Machina    metal      -0.165   1440827884   6.00   
0             First American    metal      -0.280   1436396821   1.66   
0                Cheet Sheet    metal       0.170   1427413150   3.00   
0             Beastly Bonnet    metal       0.165   1428903712   2.66   
0           Killer Exclusive    metal       0.330   1454077322  12.33   
0   Lacking Moral Fiber Mask    metal       0.500   1437526621   7.00   
0            Atomic Accolade    metal       0.220   1426809056   1.66   
0           Neon Annihilator    metal       0.660   1451956853   1.66   
0                  Fan O'War    metal       1.000   1452524306   5.00   
0                     Bolgan    metal       0.120   1432990854   5.00   
0   Crown of the Old Kingdom    metal      -0.340   1437501666   8.66   
0          That '70s Chapeau    metal       0.330   1453572779   7.33   
0       Die Regime-Panzerung    metal      -0.385   1449861302   6.00   
0                  Lone Star    metal       0.840   1444069654   4.00   
0     Stovepipe Sniper Shako    metal      -0.170   1434987629   1.66   
0              Nanobalaclava    metal      -0.215   1436284480   3.00   
0              Tyrant's Helm    metal       0.330   1453572777   7.33   
0           Hero's Hachimaki    metal      -0.340   1447222003   7.66   
0             Fortune Hunter    metal      -0.225   1439779164   2.66   
0            Hitt Mann Badge    metal       0.670   1426182938   2.00   
0         Noise Maker - Koto    metal      -0.330   1440723692   4.00   
0   Respe

### Price of a specific item (Genuine Camera Beard) -- check the price here: http://backpack.tf/stats/Genuine/Camera%20Beard/Tradable/Craftable

In [92]:
beardvalue = df[df['Name'] == 'Camera Beard']['Value']
print('Genuine Camera Beard: %.2f refined metal' %beardvalue)

Genuine Camera Beard: 5.66 refined metal


### Average price of a genuine item:

In [93]:
meanVal = df['Value'].mean()
print('Average price: %.4f refined metal' %meanVal)

Average price: 4.6851 refined metal


### Append this to a file:

In [100]:
writedf = pd.DataFrame()
writedf['DateTime'] = datetime.datetime.now()
writedf['Value (Ref)'] = beardvalue
writedf['Average Genuine Price (Ref)'] = meanVal
with open('CameraBeard.csv', 'a') as f:
    writedf.to_csv(f, header=False)
    
writedf

DateTime  Value (Ref)  Average Genuine Price (Ref)
0      NaT         5.66                     4.685069